<a href="https://colab.research.google.com/github/chhatarshal/MachineLearningPractice/blob/dev/001_Introduction_to_NLP_Fundamentals_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/chhatarshal/MachineLearningPractice/blob/main/tensorflow/helper/helper_functions.py

Lets get our helper function file using wget. We need this file to do some common task like unzip

In [2]:
!wget https://raw.githubusercontent.com/chhatarshal/MachineLearningPractice/main/tensorflow/helper/helper_functions.py

--2021-07-18 09:26:29--  https://raw.githubusercontent.com/chhatarshal/MachineLearningPractice/main/tensorflow/helper/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-07-18 09:26:29 (75.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



Lets now import some common function from helper functions file

In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

Now we are going to get data for our NLP research and will use unzip_data function from common functions to unzip all text data

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-07-18 09:26:32--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 173.194.210.128, 173.194.213.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-07-18 09:26:32 (101 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



Now we got data next important think we need to do is check the data visualize visualize visualize

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


I saw some NaN in keyword and location column


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) 
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


Lets see how many positive and negative sample we have

In [8]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

Here we see we have 4342 records says tweet are not disaster and rest 3271 tweets are disaster

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

We have 7613 train data and 3263 test data

Lets now visualize some random data

In [10]:
import random

In [11]:
random_index = random.randint(0, len(train_df))

In [12]:
print(random_index)

311


In [13]:
train_df.iloc[random_index]["text"]

'@KatieKatCubs you already know how this shit goes. World Series or Armageddon.'

In [14]:
for i in range(10):
  random_index = random.randint(0, len(train_df)) 
  text, target = train_df.iloc[random_index][["text", "target"]]
  print(text, '\n', target)
  print(f"\n\n")

./.....hmm 12000 Nigerian refugees repatriated from Cameroon http://t.co/96p3hUJNTj /( 
 1



.@LibertyGeek83 Something about kissing the ass of mass murderer doesn't sit right with me. @POTUS feels this is ok. http://t.co/LeJ5OnUs9Q 
 0



Big Top Burning The True Story Of An Arsonist A Missing Girl Û_ : http://t.co/QxH1H61cwD . 
 1



@smoak_queen 'I'm going to be in so much trouble.' 
 0



Would have just flattened the little midget ?? https://t.co/BhufevaGPu 
 0



If I get run over by an ambulance am I lucky? #justsaying #randomthought 
 0



@ArianaGrande  Girl you still lickin' public donuts???  Health hazard....u should be in jail. 
 0



Sinking carb consultative assembly plans could subconscious self live straight a leading way of escape: XkDrx 
 0



@rahulkanwal why  Jammu is soft target for terrorist 
 1



Read this already in '14 but it was and remains one of my favorite articles ever. ?'LetÛªs Like Demolish Laundry'? http://t.co/6suPThAece 
 0





We have to divide data between training and validation data and sklearn is best tool to do that

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=42)

In [17]:

# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [18]:

# Check the first 10 samples 
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 


Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:

Tokenziation - direct mapping of token (a token could be a word or a character) to number
Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

In [19]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)

In [20]:

len(train_sentences[0].split())

7

In [21]:
train_sentences[0]

'@mogacola @zamtriossu i screamed after hitting tweet'

In [22]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [23]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [24]:
# Fit the text vectorizer instance to the training data using the adapt() method
text_vectorizer.adapt(train_sentences)

In [25]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [26]:
# Choose a random sentence from the training dataset and tokenize it
for i in range(10):
  random_sentence = random.choice(train_sentences)
  print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
  text_vectorizer([random_sentence])

Original text:
Deepak Chopra's EPIC Twitter Meltdown http://t.co/ethgAGPy5G        

Vectorized version:
Original text:
Tell him Rebahe's going to destroy himself @Zenande_Mcfen @NDzedze #Ashes2Ashes        

Vectorized version:
Original text:
Hollywood movie about trapped miners released in #Chile http://t.co/r18aUtnLSd #ZippedNews http://t.co/CNqaE9foj6        

Vectorized version:
Original text:
APC Chieftain Tasks Dickson On N15b Floods Donation To Bayelsa http://t.co/LqGOe7psXp        

Vectorized version:
Original text:
@DJJOHNBLazE shout out blaze the hottest DJ in the Sothwest        

Vectorized version:
Original text:
Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/xSZicdWxq0        

Vectorized version:
Original text:
The far right racist #AvigdorLiberman calls for destruction of #Susiya ! Previously he also called for beheadings!   http://t.co/Li8otXt8hh        

Vectorized version:
Original text:
This past week has been an absolute whirlwin

In [27]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # the most common words in the vocab
bottom_5_words = words_in_vocab[-5:] # the least common words in the vocab
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [28]:
# how long is our vocab?
len(words_in_vocab)

10000

In [29]:
max_length,

(15,)

In [30]:

from tensorflow.keras import layers 

embedding = layers.Embedding(input_dim=max_vocab_length, # set the input shape
                             output_dim=128, # set the size of the embedding vector
                             embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                             input_length=max_length # how long is each input
                             )

embedding

In [31]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation, aka tokenization first)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/LjylxZ1fBi        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01143551, -0.02288607,  0.04690491, ...,  0.0140766 ,
          0.00870206, -0.00233158],
        [ 0.02309059, -0.02813979, -0.04449916, ..., -0.00760418,
         -0.03611662,  0.03567196],
        [-0.01888689,  0.00220738,  0.01042058, ...,  0.03267323,
          0.01109298, -0.00050738],
        ...,
        [-0.02390293,  0.04259726,  0.01960799, ..., -0.04651005,
         -0.01043107,  0.00693071],
        [-0.02828312, -0.03265904, -0.04397712, ..., -0.04044553,
         -0.01683422, -0.00982168],
        [-0.02688209,  0.0243648 , -0.00046072, ...,  0.02928836,
         -0.03411037, -0.04095925]]], dtype=float32)>

In [32]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.01143551, -0.02288607,  0.04690491,  0.04595753, -0.04173634,
        0.00234061,  0.04366041, -0.01205207, -0.02307102,  0.01802218,
       -0.03766905,  0.0402101 ,  0.01416579, -0.04032215, -0.02167161,
        0.02293874,  0.04638251,  0.03948525,  0.01229972, -0.01460163,
       -0.04586606,  0.02878208,  0.01928267, -0.00453518, -0.04330948,
       -0.03856748,  0.0414701 , -0.03294716,  0.03793224,  0.00915373,
        0.04812181,  0.03513564, -0.02173497, -0.04654524,  0.01251606,
       -0.03932728,  0.03030456,  0.00394507, -0.00593425,  0.03595808,
       -0.0266414 ,  0.00827998,  0.04300784,  0.02669774, -0.04982338,
        0.04152361, -0.0458433 ,  0.00407683, -0.00285965,  0.0170501 ,
       -0.0066749 , -0.00571454,  0.04890809, -0.03541082, -0.03947443,
        0.01020837, -0.00193359, -0.01344364, -0.04024858,  0.00231767,
        0.01616469,  0.02075118, -0.03474331,  0.02510036,  0.01123273,
        0.043647

In [33]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
                    ("clf", MultinomialNB()) # model the text converted to numbers
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

CPU times: user 252 ms, sys: 13.2 ms, total: 265 ms
Wall time: 301 ms


In [34]:
# Let's evalaute our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [35]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [36]:
# Function to evalaute: accuracy, precision, recall, F1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  ----
  y_true = true labels in the form of a 1D array
  y_pred = predicted label in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall and f1-score between y_true and y_pred.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100 # get accuracy score in percentage value
  # Calculate model precision, recall and f1 score using "weighted" avergage
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  # Create a dictionary of model results
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [37]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

Model 1: A simple dense model (feed-forward neural network)

In [38]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers 
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_sentences,
                        train_labels, 
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210718-092636
Epoch 1/5
215/215 [==============================] - 6s 26ms/step - loss: 0.6113 - accuracy: 0.6961 - val_loss: 0.5378 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.4419 - accuracy: 0.8151 - val_loss: 0.4760 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.3471 - accuracy: 0.8628 - val_loss: 0.4595 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.2848 - accuracy: 0.8902 - val_loss: 0.4638 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 5s 25ms/step - loss: 0.2375 - accuracy: 0.9114 - val_loss: 0.4800 - val_accuracy: 0.7795


In [39]:
# Evaluate model 1
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4800 - accuracy: 0.7795


[0.4799964129924774, 0.7795275449752808]

In [40]:
# Make predictions with model_1
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

array([[0.32222006],
       [0.78729934],
       [0.9980392 ],
       [0.12129608],
       [0.12184802],
       [0.9395814 ],
       [0.92095786],
       [0.99386907],
       [0.96184015],
       [0.27369115]], dtype=float32)

In [41]:
# Convert model_1 pred probs from probabilities to prediction labels
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [42]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [43]:
# Evaluate model_1 with our evaluation function
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 77.95275590551181,
 'f1': 0.7764267379950773,
 'precision': 0.7841069305867823,
 'recall': 0.7795275590551181}

In [44]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [45]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [46]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [47]:
embed_weights = model_1.get_weights()[2]

In [48]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [49]:
# Let's download the saved embeddings locally
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
Model 1: A simple dense model (feed-forward neural network)

Model 1: A simple dense model (feed-forward neural network)bold text

In [50]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [51]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [52]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers 
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [53]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_sentences,
                        train_labels, 
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210718-095704
Epoch 1/5
215/215 [==============================] - 6s 26ms/step - loss: 0.5447 - accuracy: 0.7694 - val_loss: 0.5040 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 5s 24ms/step - loss: 0.3514 - accuracy: 0.8853 - val_loss: 0.4616 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.2770 - accuracy: 0.9007 - val_loss: 0.4601 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.2335 - accuracy: 0.9190 - val_loss: 0.4730 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.2015 - accuracy: 0.9305 - val_loss: 0.4963 - val_accuracy: 0.7861


In [54]:
# Evaluate model 1
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 13ms/step - loss: 0.4963 - accuracy: 0.7861


[0.49630624055862427, 0.7860892415046692]